In [1]:
import torch
from transformers import BertTokenizer
from util.util import *
from tqdm import tqdm
import seaborn as sns

In [2]:
from gensim.models import KeyedVectors
import numpy as np

In [3]:
# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
w2vmodel = KeyedVectors.load_word2vec_format('/data/medg/misc/phuongpm/wikipedia-pubmed-and-PMC-w2v.bin', binary=True)

In [4]:
filename = "/data/medg/misc/phuongpm/"

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
# traindata = JsonDataset(filename + "train1.0.json")

In [7]:
# devdata = JsonDataset(filename + "dev1.0.json")

In [8]:
# testdata = JsonDataset(filename + 'test1.0.json')

In [9]:
# lenps = []
# lenqs = []

# for datum in tqdm(traindata.dataset[DATA_KEY]):
#     doc = remove_entity_marks(datum[DOC_KEY][TITLE_KEY] + " " + datum[DOC_KEY][CONTEXT_KEY]).replace(
#                         "\n", " ").lower()
#     lenps.append(len(tokenizer.tokenize(doc)))
#     for qa in datum[DOC_KEY][QAS_KEY]:
#         q = remove_entity_marks(qa[QUERY_KEY]).replace("\n", " ").lower()
#         lenqs.append(len(tokenizer.tokenize(q)))

In [10]:
# sns.distplot(lenps)

In [11]:
# sampledata = JsonDataset(filename + 'sample1.0.json')

### Convert to SQuad

In [69]:
sampledata = load_json(filename + 'dev1.0.json')

In [70]:
sampledata.keys()

dict_keys(['data', 'version'])

In [71]:
# sampledata[DATA_KEY][0]

In [72]:
def closestanswer(ans, cands):
    cand_dist = [(w2vmodel.wmdistance(a, c), c) for c in cands]
    cand_dist.sort()
    print(cand_dist)
    return cand_dist[0][1]

In [73]:
data = []
for datum in tqdm(sampledata[DATA_KEY][:20]):
    title = remove_entity_marks(datum[DOC_KEY][TITLE_KEY]).replace("\n", " ").lower()
    context = remove_entity_marks(datum[DOC_KEY][CONTEXT_KEY]).replace("\n", " ").lower()
    candidates = [w for w in to_entities(datum[DOC_KEY][CONTEXT_KEY]).lower().split() if w.startswith('@entity')]
    cands = {ent_to_plain(e) for e in set(candidates)}
    qas=[]
    for qa in datum[DOC_KEY][QAS_KEY]:
        print(len(datum[DOC_KEY][QAS_KEY]))
        a = ""
        answers = []
        for ans in qa[ANS_KEY]:
            if ans[ORIG_KEY] == "dataset":
                a = ans[TXT_KEY].lower()
            
#         print(' ' + a in context)
        start = 0
        while start < len(context) and start > -1:
            try:
                answer_start = context.index(' '+a, start+1)
            except:
                break
            answers.append({'answer_start': answer_start, 'text': a})
            start = answer_start
        qas.append({
            'answers':answers,
            'question':remove_entity_marks(qa[QUERY_KEY]).replace("\n", " ").lower().replace('@placeholder', 'what').replace(
    '.', '?').replace("▶ ",""),
            'id':qa[ID_KEY],
            'is_impossible':not bool(answers)
        })
    if len(qas) > 0:
        data.append({
            'title':title,
            'paragraphs':[{'context': context, 'qas':qas}]
        })  

100%|██████████| 20/20 [00:00<00:00, 405.33it/s]

17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
7
7
7
7
7
7
7
3
3
3
7
7
7
7
7
7
7
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
8
8
8
8
8
8
8
8
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
6
6
6
6
6
6
8
8
8
8
8
8
8
8
6
6
6
6
6
6
10
10
10
10
10
10
10
10
10
10
7
7
7
7
7
7
7
13
13
13
13
13
13
13
13
13
13
13
13
13
11
11
11
11
11
11
11
11
11
11
11
7
7
7
7
7
7
7
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
3
3
3
7
7
7
7
7
7
7
13
13
13
13
13
13
13
13
13
13
13
13
13


In [74]:
save = {'data':data, 'version':sampledata[VERSION_KEY]}

In [75]:
save

{'data': [{'title': 'caffey disease in neonatal period : the importance of the family !',
   'paragraphs': [{'context': "summary  a male newborn was apparently well until his second day of life , when increased irritability and a swelling in his right leg were noted . he was rooming - in with his mother since birth . on examination , a mass on the anterior surface of the right leg was noticed . the mass was firm , elongated , ill - defined , unmovable and painful at palpation . no overlying skin changes were seen . the newborn had a family history of neonatal bone swelling with resolution before the age of 2 . subsequent images showed hyperostosis in the diaphysis of the right tibia . after exclusion of other conditions such as trauma , osteomyelitis and congenital syphilis , the involvement of the tibial diaphysis , sparing the epiphyses and the benign course of the disease in family history , were indicative of caffey disease . the genetic study confirmed this diagnosis . caffey dise

In [40]:
# save_json(save,'/data/medg/misc/phuongpm/likesquad.1.json')